# Project 1B for Udacity Data Engineering Nanodegree
## Project Background: 
A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.

Your role is to create an Apache Cassandra database which can create queries on song play data to answer the questions, and wish to bring you on the project. You'll be given questions that the analytics team needs to answer and will be able to test your database by writing queries to answer those questions.

## Notebook Overview:
This script creates an Apache Cassandra database named 'project_1b', creates table that match how we want to model our Sparkify user behavior data, and processes the raw CSV data files to insert the data into our database tables.


# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_1b 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project_1b')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
drop_session_items_query = "drop table if exists session_items"
create_session_items_query = """
    create table if not exists session_items (session_id int, 
                                              item_in_session int, 
                                              artist text, 
                                              song text, 
                                              length decimal, 
                                              PRIMARY KEY (session_id, item_in_session))
"""
try:
    session.execute(drop_session_items_query)
    session.execute(create_session_items_query)
except Exception as e:
    print(e)                   

In [1]:
# Parse CSV to insert data into session_items table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into session_items (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

NameError: name 'csv' is not defined

In [ ]:
## SELECT statement to verify the data was entered into the table
query = "select * from session_items WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song, row.length)

## Query 2

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
drop_user_sessions_query = "drop table if exists user_sessions"
create_user_sessions_query = """
    create table if not exists user_sessions (user_id int,
                                              session_id int, 
                                              item_in_session int, 
                                              artist text, 
                                              song text, 
                                              user_name text, 
                                              PRIMARY KEY (session_id, user_id, item_in_session))
"""
try:
    session.execute(drop_user_sessions_query)
    session.execute(create_user_sessions_query)
except Exception as e:
    print(e)        

                    

In [ ]:
# Parse CSV to insert data into user_sessions table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into user_sessions (user_id, session_id, item_in_session, artist, song, user_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

In [ ]:
## SELECT statement to verify the data was entered into the user_sessions table
query = "select * from user_sessions WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id, row.session_id, row.item_in_session, row.artist, row.song, row.user_name)

## Query 3

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_song_listeners_query = "drop table if exists song_listeners"
create_song_listeners_query = """
    create table if not exists song_listeners (song text, 
                                               user_name text, 
                                               PRIMARY KEY (song, user_name))
"""
try:
    session.execute(drop_song_listeners_query)
    session.execute(create_song_listeners_query)
except Exception as e:
    print(e)          

                    

In [ ]:
# Parse CSV to insert data into song_listeners table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into song_listeners (song, user_name)"
        query = query + " VALUES (%s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1] + " " + line[4]))

In [ ]:
## SELECT statement to verify the data was entered into the song_listeners table
query = "select * from song_listeners WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.user_name)

### Drop the tables before closing out the sessions

In [ ]:
## Drop the tables before closing out the sessions
try:
    session.execute(drop_session_items_query)
    session.execute(drop_user_sessions_query)
    session.execute(drop_song_listeners_query)
except Exception as e:
    print(e)  

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()